O objetivo desse código é extrair:
1. Assunto (ou ementa);
2. acusados;
3. nome do relator;
4. e número do processo

de cada um dos PAS da CVM cujas decisões foram anunciadas entre 2000 e 2023.

Primeiro, vamos selecionar ao acaso 20 processos. Esses 20 vão ser os que vão basear o código, para podermos rodar o código de maneira mais veloz, já que ele é um código um pouco denso e trabalhoso.

In [3]:
import os
import fitz  # PyMuPDF para lidar com PDFs
import pandas as pd
import re
import random
import shutil
import pdfplumber
from PyPDF2 import PdfReader
import numpy as np
import time

In [3]:
# Caminhos
source_folder = "T:/T0mas/Faculdade/Bolsa PUB/Data"  # Pasta onde estão todas as subpastas com PDFs
test_folder = "T:/T0mas/Faculdade/Bolsa PUB/Data/Teste"  # Nova pasta para os arquivos de teste
num_samples = 20  # Número de arquivos a serem sorteados
seed = 42  # Seed para resultados reprodutíveis

# Configurar a seed
random.seed(seed)

# Criar a pasta de destino, se não existir
os.makedirs(test_folder, exist_ok=True)

# Coletar todos os caminhos dos arquivos PDF nas subpastas
all_pdfs = []
for root, dirs, files in os.walk(source_folder):
    for file in files:
        if file.endswith(".pdf"):
            all_pdfs.append(os.path.join(root, file))

# Garantir que há PDFs suficientes para sortear
if len(all_pdfs) < num_samples:
    print(f"Há apenas {len(all_pdfs)} arquivos PDF disponíveis. Todos serão copiados.")
    num_samples = len(all_pdfs)

# Sortear e copiar os PDFs
selected_pdfs = random.sample(all_pdfs, num_samples)
for pdf_path in selected_pdfs:
    shutil.copy(pdf_path, test_folder)

print(f"{len(selected_pdfs)} arquivos PDF copiados para {test_folder}")

SameFileError: 'T:/T0mas/Faculdade/Bolsa PUB/Data\\Teste\\RJ-2008-05752.pdf' and 'T:/T0mas/Faculdade/Bolsa PUB/Data/Teste\\RJ-2008-05752.pdf' are the same file

Agora, vamos pegar só o primeiro PDF da pasta (first_pdf). A partir dele vamos fazer uma primeira experimentação com o código.

In [4]:
# Caminho para a pasta "Teste"
test_folder = "T:/T0mas/Faculdade/Bolsa PUB/Data/Teste"

# Listar os arquivos PDF na pasta
pdf_files = [f for f in os.listdir(test_folder) if f.endswith(".pdf")]
if not pdf_files:
    print("Nenhum arquivo PDF encontrado na pasta Teste.")
else:
    first_pdf = os.path.join(test_folder, pdf_files[0])  # Primeiro PDF
    print(f"Processando arquivo: {first_pdf}")

Processando arquivo: T:/T0mas/Faculdade/Bolsa PUB/Data/Teste\19957-004381-2021-68.pdf


Nessa primeira célula, vamos tentar extrair o número do processo com base somente no que está dentro do pdf.

Usando PdfReader:

In [5]:
# Abrir o PDF
reader = PdfReader(first_pdf)

# Concatenar texto das duas primeiras páginas
text = ""
for i in range(2):  # Processa as duas primeiras páginas
    text += reader.pages[i].extract_text()

# Remover rodapés comuns
text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
text = re.sub(r"https:\/\/.*\n", "", text)  # Remove URLs do rodapé

# Capturar seções com regex
pattern_processo = r"PROCESSO ADMINISTRATIVO SANCIONADOR CVM nº (.*)"
pattern_data_julgamento = r"Data do julgamento:\s*(\d{2}/\d{2}/\d{4})"
pattern_acusados = r"Acusados?:\s*(.+?)(?=\n\s*(Assunto|Ementa):)"
pattern_assunto = r"(?:Assunto|Ementa):\s*(.+?)(?=\n\s*Decisão:)"
pattern_decisao = r"Decisão:\s*(.+?)(?=\n\s*(?:Presentes os procuradores|Participaram desta Sessão|Documento assinado))"

match_processo = re.search(pattern_processo, text)
match_data_julgamento = re.search(pattern_data_julgamento, text)
match_acusados = re.search(pattern_acusados, text, re.DOTALL)
match_assunto = re.search(pattern_assunto, text, re.DOTALL)
match_decisao = re.search(pattern_decisao, text, re.DOTALL)

# Obter os valores e tratar quebras de linha
numero_processo = match_processo.group(1).strip() if match_processo else np.nan
data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
acusados = match_acusados.group(1).strip() if match_acusados else np.nan
assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

# Exibir os resultados
print(f"ID: {numero_processo}")
print(f"Data do julgamento: {data_julgamento}")
print(f"Acusados: {acusados}")
print(f"Assunto: {assunto}")
print(f"Decisão: {decisao}")

ID: 19957.004381/2021-
Data do julgamento: 11/04/2023
Acusados: Orla Distribuidora de Títulos e Valores Mobiliários S.A.
Paulo Dominguez Landeira
Sagres Investimentos Administração de Recursos Ltda.
Expedito Pereira de Araujo Junior
Monetar Distribuidora de Títulos Ltda.
Marcelo de Macedo Soares e Silva
Assunto: Apurar suposta infração por parte de gestora e de administradoras de FIDC (i) ao art. 15, § 2º, da Instrução CVM n° 356/2001 c/c art. 92, caput, I, da Instrução CVM nº 555/2014; e (ii) ao art. 1º, § 1º, da Instrução CVM nº 444/2006, c/c art. 92, caput, I, da Instrução CVM nº 555/2014.  Multas e absolvições.
Decisão: Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável,  por unanimidade  de votos, decidiu:    (i) Pela condenação da  Orla Distribuidora de Títulos e Valores Mobiliários S.A. , à penalidade de  multa pecuniária , no valor de R$800.000,00 (oitocentos mil de reais), por infração 

Usando pdfplumber:

In [6]:
# Abrir o PDF usando pdfplumber
with pdfplumber.open(first_pdf) as pdf:
    # Concatenar texto das duas primeiras páginas
    text = ""
    for i in range(2):  # Processa as duas primeiras páginas
        page = pdf.pages[i]
        text += page.extract_text()

# Remover rodapés comuns
text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
text = re.sub(r"https:\/\/.*\n", "", text)  # Remove URLs do rodapé

# Capturar seções com regex
pattern_processo = r"PROCESSO ADMINISTRATIVO SANCIONADOR CVM nº (.*)"
pattern_data_julgamento = r"Data do julgamento:\s*(\d{2}/\d{2}/\d{4})"
pattern_acusados = r"Acusados?:\s*(.+?)(?=\n\s*(Assunto|Ementa):)"
pattern_assunto = r"(?:Assunto|Ementa):\s*(.+?)(?=\n\s*Decisão:)"
pattern_decisao = r"Decisão:\s*(.+?)(?=\n\s*(?:Presentes os procuradores|Participaram desta Sessão|Documento assinado))"

match_processo = re.search(pattern_processo, text)
match_data_julgamento = re.search(pattern_data_julgamento, text)
match_acusados = re.search(pattern_acusados, text, re.DOTALL)
match_assunto = re.search(pattern_assunto, text, re.DOTALL)
match_decisao = re.search(pattern_decisao, text, re.DOTALL)

# Obter os valores e tratar quebras de linha
numero_processo = match_processo.group(1).strip() if match_processo else np.nan
data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
acusados = match_acusados.group(1).strip() if match_acusados else np.nan
assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

# Exibir os resultados
print(f"ID: {numero_processo}")
print(f"Data do julgamento: {data_julgamento}")
print(f"Acusados: {acusados}")
print(f"Assunto: {assunto}")
print(f"Decisão: {decisao}")

ID: 19957.004381/2021-
Data do julgamento: 11/04/2023
Acusados: Orla Distribuidora de Títulos e Valores Mobiliários S.A.
Paulo Dominguez Landeira
Sagres Investimentos Administração de Recursos Ltda.
Expedito Pereira de Araujo Junior
Monetar Distribuidora de Títulos Ltda.
Marcelo de Macedo Soares e Silva
Assunto: Apurar suposta infração por parte de gestora e de administradoras de FIDC (i) ao art. 15, § 2º, da Instrução CVM n° 356/2001 c/c art. 92, caput, I, da Instrução CVM nº 555/2014; e (ii) ao art. 1º, § 1º, da Instrução CVM nº 444/2006, c/c art. 92, caput, I, da Instrução CVM nº 555/2014. Multas e absolvições.
Decisão: Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, por unanimidade de votos, decidiu: (i) Pela condenação da Orla Distribuidora de Títulos e Valores Mobiliários S.A., à penalidade de multa pecuniária, no valor de R$800.000,00 (oitocentos mil de reais), por infração art. 92, I

Vimos que tivemos erros. Por isso, vale a pena tentar extrair com base no nome do arquivo. É isso que fazemos agora:

Usando PyPDF2:

In [7]:
# Obter o nome do arquivo
nome_arquivo = os.path.basename(first_pdf)

# Definir o padrão de regex para os diferentes formatos de número do processo
pattern_processo_nome_arquivo = r"(?:\d{5}-\d{6}-\d{4}-\d{2}|IA-\d{4}-\d{2}|RJ-\d{4}-\d{5}|SP-\d{4}-\d{5})"

# Extrair o número do processo a partir do nome do arquivo
match_processo_nome_arquivo = re.search(pattern_processo_nome_arquivo, nome_arquivo)
numero_processo = match_processo_nome_arquivo.group(0) if match_processo_nome_arquivo else np.nan

# Abrir o PDF
reader = PdfReader(first_pdf)

# Concatenar texto das duas primeiras páginas
text = ""
for i in range(2):  # Processa as duas primeiras páginas
    text += reader.pages[i].extract_text()

# Remover rodapés comuns
text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
text = re.sub(r"https:\/\/.*\n", "", text)  # Remove URLs do rodapé

# Capturar seções com regex
pattern_data_julgamento = r"Data do julgamento:\s*(\d{2}/\d{2}/\d{4})"
pattern_acusados = r"(?:Acusados?|Interessados?|Indiciados?):\s*(.+?)(?=\n\s*(Assunto|Ementa|Relator):)"
pattern_assunto = r"(?:Assunto|Ementa):\s*(.+?)(?=\n\s*Decisão:)"
pattern_decisao = r"Decisão:\s*(.+?)(?=\n\s*(?:Presentes os procuradores|Participaram desta Sessão|Documento assinado))"

match_data_julgamento = re.search(pattern_data_julgamento, text)
match_acusados = re.search(pattern_acusados, text, re.DOTALL)
match_assunto = re.search(pattern_assunto, text, re.DOTALL)
match_decisao = re.search(pattern_decisao, text, re.DOTALL)

# Obter os valores e tratar quebras de linha
data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
acusados = match_acusados.group(1).strip() if match_acusados else np.nan
assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

# Exibir os resultados
print(f"ID: {numero_processo}")
print(f"Data do julgamento: {data_julgamento}")
print(f"Acusados: {acusados}")
print(f"Assunto: {assunto}")
print(f"Decisão: {decisao}")

ID: 19957-004381-2021-68
Data do julgamento: 11/04/2023
Acusados: Orla Distribuidora de Títulos e Valores Mobiliários S.A.
Paulo Dominguez Landeira
Sagres Investimentos Administração de Recursos Ltda.
Expedito Pereira de Araujo Junior
Monetar Distribuidora de Títulos Ltda.
Marcelo de Macedo Soares e Silva
Assunto: Apurar suposta infração por parte de gestora e de administradoras de FIDC (i) ao art. 15, § 2º, da Instrução CVM n° 356/2001 c/c art. 92, caput, I, da Instrução CVM nº 555/2014; e (ii) ao art. 1º, § 1º, da Instrução CVM nº 444/2006, c/c art. 92, caput, I, da Instrução CVM nº 555/2014.  Multas e absolvições.
Decisão: Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável,  por unanimidade  de votos, decidiu:    (i) Pela condenação da  Orla Distribuidora de Títulos e Valores Mobiliários S.A. , à penalidade de  multa pecuniária , no valor de R$800.000,00 (oitocentos mil de reais), por infraçã

Usando pdfplumber:

In [8]:
# Obter o nome do arquivo
nome_arquivo = os.path.basename(first_pdf)

# Definir o padrão de regex para os diferentes formatos de número do processo
pattern_processo_nome_arquivo = r"(?:\d{5}-\d{6}-\d{4}-\d{2}|IA-\d{4}-\d{2}|RJ-\d{4}-\d{5}|SP-\d{4}-\d{5})"

# Extrair o número do processo a partir do nome do arquivo
match_processo_nome_arquivo = re.search(pattern_processo_nome_arquivo, nome_arquivo)
numero_processo = match_processo_nome_arquivo.group(0) if match_processo_nome_arquivo else np.nan

# Abrir o PDF usando pdfplumber
with pdfplumber.open(first_pdf) as pdf:
    # Concatenar texto das duas primeiras páginas
    text = ""
    for i in range(2):  # Processa as duas primeiras páginas
        page = pdf.pages[i]
        text += page.extract_text()

# Remover rodapés comuns
text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
text = re.sub(r"https:\/\/.*\n", "", text)  # Remove URLs do rodapé

# Capturar seções com regex
pattern_data_julgamento = r"Data do julgamento:\s*(\d{2}/\d{2}/\d{4})"
pattern_acusados = r"(?:Acusados?|Interessados?|Indiciados?):\s*(.+?)(?=\n\s*(Assunto|Ementa|Relator):)"
pattern_assunto = r"(?:Assunto|Ementa):\s*(.+?)(?=\n\s*Decisão:)"
pattern_decisao = r"Decisão:\s*(.+?)(?=\n\s*(?:Presentes os procuradores|Participaram desta Sessão|Documento assinado))"

match_data_julgamento = re.search(pattern_data_julgamento, text)
match_acusados = re.search(pattern_acusados, text, re.DOTALL)
match_assunto = re.search(pattern_assunto, text, re.DOTALL)
match_decisao = re.search(pattern_decisao, text, re.DOTALL)

# Obter os valores e tratar quebras de linha
data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
acusados = match_acusados.group(1).strip() if match_acusados else np.nan
assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

# Exibir os resultados
print(f"ID: {numero_processo}")
print(f"Data do julgamento: {data_julgamento}")
print(f"Acusados: {acusados}")
print(f"Assunto: {assunto}")
print(f"Decisão: {decisao}")


ID: 19957-004381-2021-68
Data do julgamento: 11/04/2023
Acusados: Orla Distribuidora de Títulos e Valores Mobiliários S.A.
Paulo Dominguez Landeira
Sagres Investimentos Administração de Recursos Ltda.
Expedito Pereira de Araujo Junior
Monetar Distribuidora de Títulos Ltda.
Marcelo de Macedo Soares e Silva
Assunto: Apurar suposta infração por parte de gestora e de administradoras de FIDC (i) ao art. 15, § 2º, da Instrução CVM n° 356/2001 c/c art. 92, caput, I, da Instrução CVM nº 555/2014; e (ii) ao art. 1º, § 1º, da Instrução CVM nº 444/2006, c/c art. 92, caput, I, da Instrução CVM nº 555/2014. Multas e absolvições.
Decisão: Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, por unanimidade de votos, decidiu: (i) Pela condenação da Orla Distribuidora de Títulos e Valores Mobiliários S.A., à penalidade de multa pecuniária, no valor de R$800.000,00 (oitocentos mil de reais), por infração art. 92,

Agora, vamos criar um DF com os dados que extraímos.

In [9]:
# Criar um dataframe vazio com as colunas apropriadas
df = pd.DataFrame(columns=["ID","Data","Acusados", "Assunto", "Decisão"])

# Adicionar os dados extraídos a uma nova linha no dataframe
# Dividir os acusados por quebras de linha '\n' para obter uma lista de strings
acusados_lista = [acusado.strip() for acusado in acusados.split("\n")] if acusados != np.nan else []
nova_linha = pd.DataFrame({
    "ID": [numero_processo],
    "Data": [data_julgamento],
    "Acusados": [acusados_lista],
    "Assunto": [assunto],
    "Decisão": [decisao]
})

# Concatenar a nova linha ao dataframe
df = pd.concat([df, nova_linha], ignore_index=True)

# Exibir o dataframe
df

,ID,Data,Acusados,Assunto,Decisão
0,19957-004381-2021-68,11/04/2023,[Orla Distribuidora de Títulos e Valores Mobil...,Apurar suposta infração por parte de gestora e...,"Vistos, relatados e discutidos os autos, o Col..."


Agora que fizemos para um PDF, vamos testar para os 20 que estão dentro da pasta dos sorteados. Fazemos um loop.

In [10]:
# Função para extrair as informações de cada PDF
def extrair_informacoes_pdf(caminho_pdf):
    
    # Obter o nome do arquivo
    nome_arquivo = os.path.basename(caminho_pdf)

    # Definir o padrão de regex para os diferentes formatos de número do processo
    pattern_processo_nome_arquivo = r"(?:\d{5}-\d{6}-\d{4}-\d{2}|IA-\d{4}-\d{2}|RJ-\d{4}-\d{5}|SP-\d{4}-\d{5})"

    # Extrair o número do processo a partir do nome do arquivo
    match_processo_nome_arquivo = re.search(pattern_processo_nome_arquivo, nome_arquivo)
    numero_processo = match_processo_nome_arquivo.group(0) if match_processo_nome_arquivo else np.nan
    
    # Abrir o PDF
    reader = PdfReader(caminho_pdf)
    
    # Concatenar texto das duas primeiras páginas
    text = ""
    for i in range(2):  # Processa as duas primeiras páginas
        text += reader.pages[i].extract_text()

    # Remover rodapés comuns
    text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
    text = re.sub(r"https:\/\/.*\n", "", text)  # Remove URLs do rodapé

    # Capturar seções com regex
    pattern_data_julgamento = r"Data do julgamento:\s*(\d{2}/\d{2}/\d{4})"
    pattern_acusados = r"(?:Acusados?|Interessados?|Indiciados?):\s*(.+?)(?=\n\s*(Assunto|Ementa|Relator))"
    pattern_assunto = r"(?:Assunto|Ementa):\s*(.+?)(?=\n\s*Decisão:)"
    pattern_decisao = r"Decisão:\s*(.+?)(?=\n\s*(?:Presentes os procuradores|Participaram desta Sessão|Documento assinado))"

    match_data_julgamento = re.search(pattern_data_julgamento, text)
    match_acusados = re.search(pattern_acusados, text, re.DOTALL)
    match_assunto = re.search(pattern_assunto, text, re.DOTALL)
    match_decisao = re.search(pattern_decisao, text, re.DOTALL)

    # Obter os valores e tratar quebras de linha
    data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
    acusados = match_acusados.group(1).strip() if match_acusados else np.nan
    assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
    decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

    return numero_processo, data_julgamento, acusados, assunto, decisao

# Caminho para a pasta com os PDFs
pasta_pdfs = test_folder  # Altere para o caminho da sua pasta

# Criar um dataframe vazio com as colunas apropriadas
df = pd.DataFrame(columns=["ID", "Data", "Acusados", "Assunto", "Decisão"])

# Loop para processar todos os arquivos PDF na pasta
for arquivo in os.listdir(pasta_pdfs):
    if arquivo.endswith('.pdf'):  # Verifica se é um PDF
        caminho_pdf = os.path.join(pasta_pdfs, arquivo)
        
        # Extrair informações do PDF
        numero_processo, data_julgamento, acusados, assunto, decisao = extrair_informacoes_pdf(caminho_pdf)
        
        # Dividir os acusados por quebras de linha '\n' para obter uma lista de strings
        acusados_lista = [acusado.strip() for acusado in acusados.split("\n")] if isinstance(acusados, str) else []
        
        # Adicionar os dados extraídos ao dataframe
        nova_linha = pd.DataFrame({
            "ID": [numero_processo],
            "Data": [data_julgamento],
            "Acusados": [acusados_lista],
            "Assunto": [assunto],
            "Decisão": [decisao]
        })

        # Concatenar a nova linha ao dataframe
        df = pd.concat([df, nova_linha], ignore_index=True)

# Exibir o dataframe com todos os resultados
df

,ID,Data,Acusados,Assunto,Decisão
0,19957-004381-2021-68,11/04/2023,[Orla Distribuidora de Títulos e Valores Mobil...,Apurar suposta infração por parte de gestora e...,"Vistos, relatados e discutidos os autos, o Col..."
1,19957-005044-2020-15,21/12/2021,[Maverick Holding S.A.],"Responsabilidade da Maverick Holding S.A., na ...","Vistos, relatados e discutidos os autos, o Col..."
2,19957-006406-2016-09,18/06/2019,"[Letícia Ferreira Duarte do Valle, Le Valle Ag...",Prática de operação fraudulenta no mercado de ...,"Vistos, relatados e discutidos os autos, o Col..."
3,19957-008445-2016-32,18/02/2020,[Pedro Aparecido Ciriello],Realização de oferta pública de contratos de i...,"Vistos, relatados e discutidos os autos, o Col..."
4,19957-009863-2018-17,11/10/2022,[M. Invest Planejamento e Administração de Sho...,Responsabilidade da M. Invest e de seus admini...,"Vistos, relatados e discutidos os autos, o Col..."
5,IA-1997-17,NaN,"[Cid Meirelles Ferreira, Luiz Carlos de Lucca,...","Inobservância do art. 3º, da Instrução CVM nº ...",NaN
6,IA-2000-06,NaN,"[Sociedade Técnica de Auditoria – SOMATEC S/C,...",NaN,NaN
7,IA-2000-16,NaN,[],NaN,NaN
8,IA-2003-18,NaN,"[Allan Humberto de Mello, Alsino de Souza, App...",Apuração de irregularidades relacionadas à emi...,NaN
9,IA-2004-09,NaN,"[Bernardo Clemente da Fonseca Neto, Carlos Cia...",Utilização de artifício destinado a induzir e ...,NaN


Usando pdfplumber:

In [11]:
# Função para extrair as informações de cada PDF
def extrair_informacoes_pdf(caminho_pdf):
    # Obter o nome do arquivo
    nome_arquivo = os.path.basename(caminho_pdf)

    # Definir o padrão de regex para os diferentes formatos de número do processo
    pattern_processo_nome_arquivo = r"(?:\d{5}-\d{6}-\d{4}-\d{2}|IA-\d{4}-\d{2}|RJ-\d{4}-\d{5}|SP-\d{4}-\d{5})"

    # Extrair o número do processo a partir do nome do arquivo
    match_processo_nome_arquivo = re.search(pattern_processo_nome_arquivo, nome_arquivo)
    numero_processo = match_processo_nome_arquivo.group(0) if match_processo_nome_arquivo else np.nan

    # Abrir o PDF usando pdfplumber
    with pdfplumber.open(caminho_pdf) as pdf:
        # Concatenar texto das duas primeiras páginas
        text = ""
        for i in range(min(2, len(pdf.pages))):  # Verifica se há ao menos 2 páginas no PDF
            page = pdf.pages[i]
            text += page.extract_text()

    # Remover rodapés comuns
    text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
    text = re.sub(r"https:\/\/.*\n", "", text)  # Remove URLs do rodapé

    # Capturar seções com regex
    pattern_data_julgamento = r"Data do julgamento:\s*(\d{2}/\d{2}/\d{4})"
    pattern_acusados = r"(?:Acusados?|Interessados?|Indiciados?):\s*(.+?)(?=\n\s*(Assunto|Ementa|Relator))"
    pattern_assunto = r"(?:Assunto|Ementa):\s*(.+?)(?=\n\s*Decisão:)"
    pattern_decisao = r"Decisão:\s*(.+?)(?=\n\s*(?:Presentes os procuradores|Participaram desta Sessão|Documento assinado))"

    match_data_julgamento = re.search(pattern_data_julgamento, text)
    match_acusados = re.search(pattern_acusados, text, re.DOTALL)
    match_assunto = re.search(pattern_assunto, text, re.DOTALL)
    match_decisao = re.search(pattern_decisao, text, re.DOTALL)

    # Obter os valores e tratar quebras de linha
    data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
    acusados = match_acusados.group(1).strip() if match_acusados else np.nan
    assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
    decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

    return numero_processo, data_julgamento, acusados, assunto, decisao

# Caminho para a pasta com os PDFs
pasta_pdfs = test_folder  # Altere para o caminho da sua pasta

# Criar um dataframe vazio com as colunas apropriadas
df = pd.DataFrame(columns=["ID", "Data", "Acusados", "Assunto", "Decisão"])

# Loop para processar todos os arquivos PDF na pasta
for arquivo in os.listdir(pasta_pdfs):
    if arquivo.endswith('.pdf'):  # Verifica se é um PDF
        caminho_pdf = os.path.join(pasta_pdfs, arquivo)
        
        # Extrair informações do PDF
        numero_processo, data_julgamento, acusados, assunto, decisao = extrair_informacoes_pdf(caminho_pdf)
        
        # Dividir os acusados por quebras de linha '\n' para obter uma lista de strings
        acusados_lista = [acusado.strip() for acusado in acusados.split("\n")] if isinstance(acusados, str) else []
        
        # Adicionar os dados extraídos ao dataframe
        nova_linha = pd.DataFrame({
            "ID": [numero_processo],
            "Data": [data_julgamento],
            "Acusados": [acusados_lista],
            "Assunto": [assunto],
            "Decisão": [decisao]
        })

        # Concatenar a nova linha ao dataframe
        df = pd.concat([df, nova_linha], ignore_index=True)

# Exibir o dataframe com todos os resultados
df

,ID,Data,Acusados,Assunto,Decisão
0,19957-004381-2021-68,11/04/2023,[Orla Distribuidora de Títulos e Valores Mobil...,Apurar suposta infração por parte de gestora e...,"Vistos, relatados e discutidos os autos, o Col..."
1,19957-005044-2020-15,21/12/2021,[Maverick Holding S.A.],"Responsabilidade da Maverick Holding S.A., na ...","Vistos, relatados e discutidos os autos, o Col..."
2,19957-006406-2016-09,18/06/2019,"[Letícia Ferreira Duarte do Valle, Le Valle Ag...",Prática de operação fraudulenta no mercado de ...,"Vistos, relatados e discutidos os autos, o Col..."
3,19957-008445-2016-32,18/02/2020,[Pedro Aparecido Ciriello],Realização de oferta pública de contratos de i...,"Vistos, relatados e discutidos os autos, o Col..."
4,19957-009863-2018-17,11/10/2022,[M. Invest Planejamento e Administração de Sho...,Responsabilidade da M. Invest e de seus admini...,"Vistos, relatados e discutidos os autos, o Col..."
5,IA-1997-17,NaN,"[Cid Meirelles Ferreira, Luiz Carlos de Lucca,...","Inobservância do art. 3º, da Instrução CVM nº ...",NaN
6,IA-2000-06,NaN,"[Sociedade Técnica de Auditoria – SOMATEC S/C,...",NaN,NaN
7,IA-2000-16,NaN,[],NaN,NaN
8,IA-2003-18,NaN,"[Allan Humberto de Mello, Alsino de Souza, App...",Apuração de irregularidades relacionadas à emi...,NaN
9,IA-2004-09,NaN,"[Bernardo Clemente da Fonseca Neto, Carlos Cia...",Utilização de artifício destinado a induzir e ...,NaN


Vejam: os problemas nas colunas de "Assunto" e "Decisão" são esperados. Apesar disso, é muito importante conseguimos captar todos os Acusados. Vemos que o problema está na expressão regular que usamos para identificá-los. Além de "Acusados", as vezes os diretores usam "Interessados" ou "Indiciados."

In [12]:
pd.set_option('display.max_colwidth', None)

df["Acusados"]

0                                                                                                          [Orla Distribuidora de Títulos e Valores Mobiliários S.A., Paulo Dominguez Landeira, Sagres Investimentos Administração de Recursos Ltda., Expedito Pereira de Araujo Junior, Monetar Distribuidora de Títulos Ltda., Marcelo de Macedo Soares e Silva]
1                                                                                                                                                                                                                                                                                                                                          [Maverick Holding S.A.]
2                                                                                                                                                                                                                                                                          [Letícia Ferreira Duart

Percebemos que, 90% das vezes, a decisão termina com a data, ou com a expressão "30 dias", ou com a data. Vamos readaptar a parte de decisão para captar isso.

Sobre a parte de Assunto: vamos modficar de forma que não seja necessária uma quebra de linha.

In [13]:
# Função para extrair as informações de cada PDF
def extrair_informacoes_pdf(caminho_pdf):
    # Obter o nome do arquivo
    nome_arquivo = os.path.basename(caminho_pdf)

    # Definir o padrão de regex para os diferentes formatos de número do processo
    pattern_processo_nome_arquivo = r"(?:\d{5}-\d{6}-\d{4}-\d{2}|IA-\d{4}-\d{2}|RJ-\d{4}-\d{5}|SP-\d{4}-\d{5})"

    # Extrair o número do processo a partir do nome do arquivo
    match_processo_nome_arquivo = re.search(pattern_processo_nome_arquivo, nome_arquivo)
    numero_processo = match_processo_nome_arquivo.group(0) if match_processo_nome_arquivo else np.nan

    # Abrir o PDF usando pdfplumber
    with pdfplumber.open(caminho_pdf) as pdf:
        # Concatenar texto das duas primeiras páginas
        text = ""
        for i in range(min(2, len(pdf.pages))):  # Verifica se há ao menos 2 páginas no PDF
            page = pdf.pages[i]
            text += page.extract_text()

    # Remover rodapés comuns
    text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
    text = re.sub(r"https:\/\/.*\n", "", text)  # Remove URLs do rodapé

    # Capturar seções com regex
    pattern_data_julgamento = r"(Data do julgamento:\s*(\d{2}/\d{2}/\d{4})|(?:Rio de Janeiro|São Paulo|Brasília),\s*(\d{2} de (?:janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de 20\d{2}))"
    pattern_acusados = r"(?:Acusados?|Interessados?|Indiciados?):\s*(.+?)(?=\s*(Assunto|Ementa|Relator))"
    pattern_assunto = r"(?:Assunto|Ementa):\s*(.+?)(?=\s*Decisão)" 
    pattern_decisao = r"(Vistos, relatados e discutidos os autos\s*(.+?))(?=\s*(?:30 dias|30 (trinta) dias|trinta dias|Participaram desta Sessão|(Rio de Janeiro|São Paulo|Brasília), \d{2} de (janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de 20\d{2}))"

    match_data_julgamento = re.search(pattern_data_julgamento, text)
    match_acusados = re.search(pattern_acusados, text, re.DOTALL)
    match_assunto = re.search(pattern_assunto, text, re.DOTALL)
    match_decisao = re.search(pattern_decisao, text, re.DOTALL)

    # Obter os valores e tratar quebras de linha
    data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
    acusados = match_acusados.group(1).strip() if match_acusados else np.nan
    assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
    decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

    return numero_processo, data_julgamento, acusados, assunto, decisao

# Caminho para a pasta com os PDFs
pasta_pdfs = test_folder  # Altere para o caminho da sua pasta

# Criar um dataframe vazio com as colunas apropriadas
df2 = pd.DataFrame(columns=["ID", "Data", "Acusados", "Assunto", "Decisão"])

# Iniciar o timer
start_time = time.time()

# Loop para processar todos os arquivos PDF na pasta
for arquivo in os.listdir(pasta_pdfs):
    if arquivo.endswith('.pdf'):  # Verifica se é um PDF
        caminho_pdf = os.path.join(pasta_pdfs, arquivo)
        
        # Extrair informações do PDF
        numero_processo, data_julgamento, acusados, assunto, decisao = extrair_informacoes_pdf(caminho_pdf)
        
        # Dividir os acusados por quebras de linha '\n' para obter uma lista de strings
        acusados_lista = [acusado.strip() for acusado in re.split(r'[\n,]', acusados) if acusado.strip()] if isinstance(acusados, str) else []
        
        # Adicionar os dados extraídos ao dataframe
        nova_linha = pd.DataFrame({
            "ID": [numero_processo],
            "Data": [data_julgamento],
            "Acusados": [acusados_lista],
            "Assunto": [assunto],
            "Decisão": [decisao]
        })

        # Concatenar a nova linha ao dataframe
        df2 = pd.concat([df2, nova_linha], ignore_index=True)

# Calcular o tempo de execução
end_time = time.time()
execution_time = end_time - start_time

# Exibir o tempo de execução
print(f"Tempo de execução: {execution_time:.2f} segundos")

# Exibir o dataframe com todos os resultados
df2

Tempo de execução: 28.64 segundos


,ID,Data,Acusados,Assunto,Decisão
0,19957-004381-2021-68,Data do julgamento: 11/04/2023,"[Orla Distribuidora de Títulos e Valores Mobiliários S.A., Paulo Dominguez Landeira, Sagres Investimentos Administração de Recursos Ltda., Expedito Pereira de Araujo Junior, Monetar Distribuidora de Títulos Ltda., Marcelo de Macedo Soares e Silva]","Apurar suposta infração por parte de gestora e de administradoras de FIDC (i) ao art. 15, § 2º, da Instrução CVM n° 356/2001 c/c art. 92, caput, I, da Instrução CVM nº 555/2014; e (ii) ao art. 1º, § 1º, da Instrução CVM nº 444/2006, c/c art. 92, caput, I, da Instrução CVM nº 555/2014. Multas e absolvições.","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, por unanimidade de votos, decidiu: (i) Pela condenação da Orla Distribuidora de Títulos e Valores Mobiliários S.A., à penalidade de multa pecuniária, no valor de R$800.000,00 (oitocentos mil de reais), por infração art. 92, I, da ICVM 555 c/c o art. 1º, §1º, da ICVM 444/2006; (ii) Pela condenação de Paulo Dominguez Landeira, na qualidade de diretor responsável da Orla, à penalidade de multa pecuniária, no valor de R$400.000,00 (quatrocentos mil reais), por infração art. 92, I, da ICVM 555 c/c o art. 1º, §1º, da ICVM 444/2006; (iii) Pela condenação da Sagres Investimentos Administração de Recursos Ltda., à penalidade de multa pecuniária, no valor de R$680.000,00 (seiscentos e oitenta mil reais), por infração art. 92, I, da ICVM 555 c/c o art. 1º, §1º, da ICVM 444/2006; (iv) Pela condenação de Expedito Pereira de Araujo Junior, na qualidade de diretor responsável da Sagres, à penalidade de multa pecuniária, no ICVM 555 c/c o art. 1º, §1º, da ICVM 444/2006; (v) Pela condenação da Monetar Distribuidora de Títulos Ltda., à penalidade de multa pecuniária, no valor de R$680.000,00 (seiscentos e oitenta mil reais), por infração art. 92, I, da ICVM 555 c/c o art. 1º, §1º, da ICVM 444/2006; (vi) Pela condenação de Marcelo de Macedo Soares e Silva, na qualidade de diretor responsável da Monetar, à penalidade de multa pecuniária, no valor de R$340.000,00 (trezentos e quarenta mil reais), por infração art. 92, I, da ICVM 555/2014 c/c o art. 1º, §1º, da ICVM 444/2006; e (vii) Pela absolvição da Orla Distribuidora de Títulos e Valores Mobiliários S.A., da Monetar Distribuidora de Títulos Ltda. e de seus diretores responsáveis, Paulo Dominguez Landeira e Marcelo de Macedo Soares e Silva, da acusação de infração art. 92, I, da ICVM 555/2014 c/c ao art. 15, § 2º, da ICVM 356/2001. Os acusados punidos terão um prazo de"
1,19957-005044-2020-15,Data do julgamento: 21/12/2021,[Maverick Holding S.A.],"Responsabilidade da Maverick Holding S.A., na qualidade de acionista controladora da Mlog S.A., por violação ao art. 106 da Lei nº 6.404/1976. Multa.","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, por unanimidade* de votos, decidiu pela condenação da Maverick, na qualidade de acionista controladora da Mlog, à penalidade de multa pecuniária no valor de R$267.750,00 (duzentos e sessenta e sete mil setecentos e cinquenta reais), por infringir o art. 106 da Lei nº 6.404/1976. O acusado punido terá um prazo de"
2,19957-006406-2016-09,Data do julgamento: 18/06/2019,"[Letícia Ferreira Duarte do Valle, Le Valle Agente Autônomo de Investimentos S/S Ltda.]","Prática de operação fraudulenta no mercado de valores mobiliários. Infração ao item I c/c o item II, letra “c”, da Instrução CVM nº 08/79. Proibição temporária e multa.","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, por unanimidade de votos, decidiu: 1. Com fundamento no art. 11, VII, da Lei nº 6.385/76, aplicar à acusada Leticia Ferreira Duarte do Valle a penalidade de proibição temporária, pelo prazo de cinco anos, de praticar toda e 

In [14]:
pd.set_option('display.max_colwidth', 150)

df2

,ID,Data,Acusados,Assunto,Decisão
0,19957-004381-2021-68,Data do julgamento: 11/04/2023,"[Orla Distribuidora de Títulos e Valores Mobiliários S.A., Paulo Dominguez Landeira, Sagres Investimentos Administração de Recursos Ltda., Expedit...","Apurar suposta infração por parte de gestora e de administradoras de FIDC (i) ao art. 15, § 2º, da Instrução CVM n° 356/2001 c/c art. 92, caput, I...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
1,19957-005044-2020-15,Data do julgamento: 21/12/2021,[Maverick Holding S.A.],"Responsabilidade da Maverick Holding S.A., na qualidade de acionista controladora da Mlog S.A., por violação ao art. 106 da Lei nº 6.404/1976. Multa.","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
2,19957-006406-2016-09,Data do julgamento: 18/06/2019,"[Letícia Ferreira Duarte do Valle, Le Valle Agente Autônomo de Investimentos S/S Ltda.]","Prática de operação fraudulenta no mercado de valores mobiliários. Infração ao item I c/c o item II, letra “c”, da Instrução CVM nº 08/79. Proibiç...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
3,19957-008445-2016-32,Data do julgamento: 18/02/2020,[Pedro Aparecido Ciriello],Realização de oferta pública de contratos de investimento coletivo sem a obtenção do devido registro na CVM previsto no art. 19 da Lei nº 6.385/76...,"Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
4,19957-009863-2018-17,Data do julgamento: 11/10/2022,"[M. Invest Planejamento e Administração de Shopping Center S.A., Cyro Santiago Rodrigues, Lorival Rodrigue]","Responsabilidade da M. Invest e de seus administradores pela não elaboração tempestiva e não divulgação de demonstrações financeiras, em infração ...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
5,IA-1997-17,"Rio de Janeiro, 19 de maio de 2005","[Cid Meirelles Ferreira, Luiz Carlos de Lucca, Mérito Auditores Independentes, Milton de Souza Meirelles Filho]","Inobservância do art. 3º, da Instrução CVM nº 10/80. - Aquisição, pela controlada, de ações da controladora em percentual superior ao permitido. I...","Vistos, relatados e discutidos os autos, o Colegiado da CVM, com base na prova dos autos e na legislação aplicável, decidiu, por unanimidade de vo..."
6,IA-2000-06,"Rio de Janeiro, 22 de agosto de 2002","[Sociedade Técnica de Auditoria – SOMATEC S/C, Zilo Matsunaga]",NaN,"Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável dec..."
7,IA-2000-16,NaN,[],NaN,NaN
8,IA-2003-18,"Rio de Janeiro, 08 de dezembro de 2005","[Allan Humberto de Mello, Alsino de Souza, Apply Auditores Associados S/C, Bruno de Mello Bomeny, Frederico Figueiredo Bomeny, Geraldo Barbosa de ...","Apuração de irregularidades relacionadas à emissão de debêntures destinadas à implementação de empreendimento hoteleiro objeto da companhia, const...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, co..."
9,IA-2004-09,"Rio de Janeiro, 20 de junho de 2006","[Bernardo Clemente da Fonseca Neto, Carlos Ciampolini, Carlos Eduardo Brito de Ávila Camargo, Indusval S.A. CTVM, João Antônio Porchat Forbes, Sér...","Utilização de artifício destinado a induzir e manter a Votorantim Comercial Exportadora e Importadora Ltda. em erro, através da distribuição irreg...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicáv

Vamos por caso:

Na linha 6, tivemos um problema com o assunto. Será que é pq temos um espaço entre "Ementa" e os dois-pontos?

Na linha 7, poderíamos ter captado ao menos os acusados. Será que é pq a expressão "Interessados" está toda em letras maiúsculas? Apesar disso, é a única coisa que poderíamos ter captado, já que o relatório está completamente desfigurado.

Na linha 12, tivemos um problema com o assunto e com os acusados.Eu acho que é pq temos, mais uma vez, espaços entre "Interessados"/"Ementa" e os dois-pontos.

Na linha 13, tivemos um problema com o assunto. Mais uma vez, a palavra "Ementa" está separada por um espaço dos dois-pontos.

Na linha 16, apesar de tudo, podemos considerar que tivemos sucesso, já que o relatório está completamente desfigurado, mais um vez, e mesmo assim conseguimos capar a informação do nome do acusado, que é de fato a única disponível no texto.

Como corrigir?

Para a linha 6: vamos reajustar de modo a possibilitar um espaço entre as expressões e os dois-pontos.

Para a linha 7: vamos adicionar as expressões em caps-lock ao conjunto de expressões regulares que vão ser capturadas.

Para a linha 12: vamos fazer como na linha 6, possibilitando um espaço entre as expressões e os dois-pontos.

Para a linha 13: mesmo que a linha 12.

Além disso, para as outras categorias, vamos manter a possibilidade de termos espaços e caps-lock, quando convém.

O novo código é:

In [16]:
# Função para extrair as informações de cada PDF
def extrair_informacoes_pdf(caminho_pdf):
    # Obter o nome do arquivo
    nome_arquivo = os.path.basename(caminho_pdf)

    # Definir o padrão de regex para os diferentes formatos de número do processo
    pattern_processo_nome_arquivo = r"(?:\d{5}-\d{6}-\d{4}-\d{2}|IA-\d{4}-\d{2}|RJ-\d{4}-\d{5}|SP-\d{4}-\d{5})"

    # Extrair o número do processo a partir do nome do arquivo
    match_processo_nome_arquivo = re.search(pattern_processo_nome_arquivo, nome_arquivo)
    numero_processo = match_processo_nome_arquivo.group(0) if match_processo_nome_arquivo else np.nan

    # Abrir o PDF usando pdfplumber
    with pdfplumber.open(caminho_pdf) as pdf:
        # Concatenar texto das duas primeiras páginas
        text = ""
        for i in range(min(2, len(pdf.pages))):  # Verifica se há ao menos 2 páginas no PDF
            page = pdf.pages[i]
            text += page.extract_text()

    # Remover rodapés comuns
    text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
    text = re.sub(r"https:\/\/.*\n", "", text)  # Remove URLs do rodapé

    # Capturar seções com regex
    pattern_data_julgamento = r"(Data do julgamento:\s*(\d{2}/\d{2}/\d{4})|(?:Rio de Janeiro|São Paulo|Brasília),\s*(\d{2} de (?:janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de 20\d{2}))"
    pattern_acusados = r"(?:Acusados?\s*:|Interessados?\s*:|Indiciados?\s*:|ACUSADOS?\s*:|INTERESSADOS?\s*:|INDICIADOS?\s*:)\s*(.+?)(?=\s*(Assunto|Ementa|Relator))"
    pattern_assunto = r"(?:Assunto\s*:|Ementa\s*:|ASSUNTO\s*:|EMENTA\s*:)\s*(.+?)(?=\s*Decisão)" 
    pattern_decisao = r"(Vistos, relatados e discutidos os autos\s*(.+?))(?=\s*(?:30 dias|30 (trinta) dias|trinta dias|Participaram desta Sessão|(Rio de Janeiro|São Paulo|Brasília), \d{2} de (janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de 20\d{2}))"

    match_data_julgamento = re.search(pattern_data_julgamento, text)
    match_acusados = re.search(pattern_acusados, text, re.DOTALL)
    match_assunto = re.search(pattern_assunto, text, re.DOTALL)
    match_decisao = re.search(pattern_decisao, text, re.DOTALL)

    # Obter os valores e tratar quebras de linha
    data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
    acusados = match_acusados.group(1).strip() if match_acusados else np.nan
    assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
    decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

    return numero_processo, data_julgamento, acusados, assunto, decisao

# Caminho para a pasta com os PDFs
pasta_pdfs = test_folder  # Altere para o caminho da sua pasta

# Criar um dataframe vazio com as colunas apropriadas
df3 = pd.DataFrame(columns=["ID", "Data", "Acusados", "Assunto", "Decisão"])

# Loop para processar todos os arquivos PDF na pasta
for arquivo in os.listdir(pasta_pdfs):
    if arquivo.endswith('.pdf'):  # Verifica se é um PDF
        caminho_pdf = os.path.join(pasta_pdfs, arquivo)
        
        # Iniciar o timer para a iteração
        iter_start_time = time.time()
        
        # Extrair informações do PDF
        numero_processo, data_julgamento, acusados, assunto, decisao = extrair_informacoes_pdf(caminho_pdf)
        
        # Dividir os acusados por quebras de linha '\n' para obter uma lista de strings
        acusados_lista = [acusado.strip() for acusado in re.split(r'[\n,]', acusados) if acusado.strip()] if isinstance(acusados, str) else []
        
        # Adicionar os dados extraídos ao dataframe
        nova_linha = pd.DataFrame({
            "ID": [numero_processo],
            "Data": [data_julgamento],
            "Acusados": [acusados_lista],
            "Assunto": [assunto],
            "Decisão": [decisao]
        })

        # Concatenar a nova linha ao dataframe
        df3 = pd.concat([df3, nova_linha], ignore_index=True)

        # Calcular o tempo da iteração
        iter_end_time = time.time()
        iter_execution_time = iter_end_time - iter_start_time

        # Exibir o tempo de execução da iteração
        print(f"Tempo para processar {arquivo}: {iter_execution_time:.2f} segundos")

# Exibir o dataframe com todos os resultados
df3

Tempo para processar 19957-004381-2021-68.pdf: 1.27 segundos
Tempo para processar 19957-005044-2020-15 (RJ-2020-03041).pdf: 0.66 segundos
Tempo para processar 19957-006406-2016-09 (RJ-2016-07486).pdf: 1.05 segundos
Tempo para processar 19957-008445-2016-32 (RJ-2016-08381).pdf: 1.04 segundos
Tempo para processar 19957-009863-2018-17 (RJ-2018-07223).pdf: 1.24 segundos
Tempo para processar IA-1997-17_A.pdf: 1.44 segundos
Tempo para processar IA-2000-06.pdf: 1.44 segundos
Tempo para processar IA-2000-16.pdf: 1.80 segundos
Tempo para processar IA-2003-18.pdf: 1.71 segundos
Tempo para processar IA-2004-09.pdf: 1.85 segundos
Tempo para processar IA-2005-19.pdf: 1.59 segundos
Tempo para processar IA-2006-15.pdf: 1.49 segundos
Tempo para processar RJ-2000-05907.pdf: 1.55 segundos
Tempo para processar RJ-2001-06094.pdf: 1.52 segundos
Tempo para processar RJ-2003-05509.pdf: 1.69 segundos
Tempo para processar RJ-2007-04107.pdf: 1.64 segundos
Tempo para processar RJ-2008-05752.pdf: 1.46 segundos
Te

,ID,Data,Acusados,Assunto,Decisão
0,19957-004381-2021-68,Data do julgamento: 11/04/2023,"[Orla Distribuidora de Títulos e Valores Mobiliários S.A., Paulo Dominguez Landeira, Sagres Investimentos Administração de Recursos Ltda., Expedit...","Apurar suposta infração por parte de gestora e de administradoras de FIDC (i) ao art. 15, § 2º, da Instrução CVM n° 356/2001 c/c art. 92, caput, I...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
1,19957-005044-2020-15,Data do julgamento: 21/12/2021,[Maverick Holding S.A.],"Responsabilidade da Maverick Holding S.A., na qualidade de acionista controladora da Mlog S.A., por violação ao art. 106 da Lei nº 6.404/1976. Multa.","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
2,19957-006406-2016-09,Data do julgamento: 18/06/2019,"[Letícia Ferreira Duarte do Valle, Le Valle Agente Autônomo de Investimentos S/S Ltda.]","Prática de operação fraudulenta no mercado de valores mobiliários. Infração ao item I c/c o item II, letra “c”, da Instrução CVM nº 08/79. Proibiç...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
3,19957-008445-2016-32,Data do julgamento: 18/02/2020,[Pedro Aparecido Ciriello],Realização de oferta pública de contratos de investimento coletivo sem a obtenção do devido registro na CVM previsto no art. 19 da Lei nº 6.385/76...,"Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
4,19957-009863-2018-17,Data do julgamento: 11/10/2022,"[M. Invest Planejamento e Administração de Shopping Center S.A., Cyro Santiago Rodrigues, Lorival Rodrigue]","Responsabilidade da M. Invest e de seus administradores pela não elaboração tempestiva e não divulgação de demonstrações financeiras, em infração ...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
5,IA-1997-17,"Rio de Janeiro, 19 de maio de 2005","[Cid Meirelles Ferreira, Luiz Carlos de Lucca, Mérito Auditores Independentes, Milton de Souza Meirelles Filho]","Inobservância do art. 3º, da Instrução CVM nº 10/80. - Aquisição, pela controlada, de ações da controladora em percentual superior ao permitido. I...","Vistos, relatados e discutidos os autos, o Colegiado da CVM, com base na prova dos autos e na legislação aplicável, decidiu, por unanimidade de vo..."
6,IA-2000-06,"Rio de Janeiro, 22 de agosto de 2002","[Sociedade Técnica de Auditoria – SOMATEC S/C, Zilo Matsunaga]",I – Desenvolvimento da atividade de auditoria independente em desacordo com as normas profissionais emanadas do Conselho Federal de Contabilidade....,"Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável dec..."
7,IA-2000-16,NaN,[],NaN,NaN
8,IA-2003-18,"Rio de Janeiro, 08 de dezembro de 2005","[Allan Humberto de Mello, Alsino de Souza, Apply Auditores Associados S/C, Bruno de Mello Bomeny, Frederico Figueiredo Bomeny, Geraldo Barbosa de ...","Apuração de irregularidades relacionadas à emissão de debêntures destinadas à implementação de empreendimento hoteleiro objeto da companhia, const...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, co..."
9,IA-2004-09,"Rio de Janeiro, 20 de junho de 2006","[Bernardo Clemente da Fonseca Neto, Carlos Ciampolini, Carlos Eduardo Brito de Ávila Camargo, Indusval S.A. CTVM, João Antônio Porchat Forbes, Sér...","Utilização de artifício destinado a induzir e manter a Votorantim Comercial Exportadora e Importadora Ltda. em erro, através da distribuição irreg

In [18]:
# Função para extrair as informações de cada PDF
def extrair_informacoes_pdf(caminho_pdf):
    nome_arquivo = os.path.basename(caminho_pdf)
    pattern_processo_nome_arquivo = r"(?:\d{5}-\d{6}-\d{4}-\d{2}|IA-\d{4}-\d{2}|RJ-\d{4}-\d{5}|SP-\d{4}-\d{5})"
    match_processo_nome_arquivo = re.search(pattern_processo_nome_arquivo, nome_arquivo)
    numero_processo = match_processo_nome_arquivo.group(0) if match_processo_nome_arquivo else np.nan

    with pdfplumber.open(caminho_pdf) as pdf:
        text = ""
        for i in range(min(2, len(pdf.pages))):
            page = pdf.pages[i]
            text += page.extract_text()

    text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
    text = re.sub(r"https:\/\/.*\n", "", text)

    pattern_data_julgamento = r"(Data do julgamento:\s*(\d{2}/\d{2}/\d{4})|(?:Rio de Janeiro|São Paulo|Brasília),\s*(\d{2} de (?:janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de 20\d{2}))"
    pattern_acusados = r"(?:Acusados?\s*:|Interessados?\s*:|Indiciados?\s*:|ACUSADOS?\s*:|INTERESSADOS?\s*:|INDICIADOS?\s*:)\s*(.+?)(?=\s*(Assunto|Ementa|Relator|ASSUNTO|EMENTA|RELATOR))"
    pattern_assunto = r"(?:Assunto\s*:|Ementa\s*:|ASSUNTO\s*:|EMENTA\s*:)\s*(.+?)(?=\s*(Decisão|DECISÃO))" 
    pattern_decisao = r"(Vistos, relatados e discutidos os autos\s*(.+?))(?=\s*(?:30 dias|30 (trinta) dias|trinta dias|Participaram desta Sessão|(Rio de Janeiro|São Paulo|Brasília), \d{2} de (janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de 20\d{2}))"

    match_data_julgamento = re.search(pattern_data_julgamento, text)
    match_acusados = re.search(pattern_acusados, text, re.DOTALL)
    match_assunto = re.search(pattern_assunto, text, re.DOTALL)
    match_decisao = re.search(pattern_decisao, text, re.DOTALL)

    data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
    acusados = match_acusados.group(1).strip() if match_acusados else np.nan
    assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
    decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

    return numero_processo, data_julgamento, acusados, assunto, decisao

# Caminho para a pasta com os PDFs
pasta_pdfs = test_folder   # Altere para o caminho da sua pasta

# Criar um dataframe vazio com as colunas apropriadas
df4 = pd.DataFrame(columns=["ID", "Data", "Acusados", "Assunto", "Decisão"])

# Iniciar o timer total
start_time = time.time()

# Loop para processar todos os arquivos PDF na pasta
for arquivo in os.listdir(pasta_pdfs):
    if arquivo.endswith('.pdf'):
        caminho_pdf = os.path.join(pasta_pdfs, arquivo)

        # Timer para cada iteração
        iter_start = time.time()

        # Extrair informações do PDF
        numero_processo, data_julgamento, acusados, assunto, decisao = extrair_informacoes_pdf(caminho_pdf)

        acusados_lista = [acusado.strip() for acusado in re.split(r'[\n,]', acusados) if acusado.strip()] if isinstance(acusados, str) else []
        
        nova_linha = pd.DataFrame({
            "ID": [numero_processo],
            "Data": [data_julgamento],
            "Acusados": [acusados_lista],
            "Assunto": [assunto],
            "Decisão": [decisao]
        })

        df4 = pd.concat([df4, nova_linha], ignore_index=True)

        # Timer de cada iteração
        iter_end = time.time()
        iter_time = iter_end - iter_start
        print(f"Tempo para processar {arquivo}: {iter_time:.2f} segundos")

# Calcular o tempo total
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo total de execução: {execution_time:.2f} segundos")

# Exibir o dataframe com todos os resultados
df4

Tempo para processar 19957-004381-2021-68.pdf: 1.40 segundos
Tempo para processar 19957-005044-2020-15 (RJ-2020-03041).pdf: 0.67 segundos
Tempo para processar 19957-006406-2016-09 (RJ-2016-07486).pdf: 0.99 segundos
Tempo para processar 19957-008445-2016-32 (RJ-2016-08381).pdf: 0.85 segundos
Tempo para processar 19957-009863-2018-17 (RJ-2018-07223).pdf: 1.24 segundos
Tempo para processar IA-1997-17_A.pdf: 1.40 segundos
Tempo para processar IA-2000-06.pdf: 1.48 segundos
Tempo para processar IA-2000-16.pdf: 1.82 segundos
Tempo para processar IA-2003-18.pdf: 1.74 segundos
Tempo para processar IA-2004-09.pdf: 1.69 segundos
Tempo para processar IA-2005-19.pdf: 1.65 segundos
Tempo para processar IA-2006-15.pdf: 1.56 segundos
Tempo para processar RJ-2000-05907.pdf: 1.61 segundos
Tempo para processar RJ-2001-06094.pdf: 1.55 segundos
Tempo para processar RJ-2003-05509.pdf: 1.67 segundos
Tempo para processar RJ-2007-04107.pdf: 1.71 segundos
Tempo para processar RJ-2008-05752.pdf: 1.55 segundos
Te

,ID,Data,Acusados,Assunto,Decisão
0,19957-004381-2021-68,Data do julgamento: 11/04/2023,"[Orla Distribuidora de Títulos e Valores Mobiliários S.A., Paulo Dominguez Landeira, Sagres Investimentos Administração de Recursos Ltda., Expedit...","Apurar suposta infração por parte de gestora e de administradoras de FIDC (i) ao art. 15, § 2º, da Instrução CVM n° 356/2001 c/c art. 92, caput, I...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
1,19957-005044-2020-15,Data do julgamento: 21/12/2021,[Maverick Holding S.A.],"Responsabilidade da Maverick Holding S.A., na qualidade de acionista controladora da Mlog S.A., por violação ao art. 106 da Lei nº 6.404/1976. Multa.","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
2,19957-006406-2016-09,Data do julgamento: 18/06/2019,"[Letícia Ferreira Duarte do Valle, Le Valle Agente Autônomo de Investimentos S/S Ltda.]","Prática de operação fraudulenta no mercado de valores mobiliários. Infração ao item I c/c o item II, letra “c”, da Instrução CVM nº 08/79. Proibiç...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
3,19957-008445-2016-32,Data do julgamento: 18/02/2020,[Pedro Aparecido Ciriello],Realização de oferta pública de contratos de investimento coletivo sem a obtenção do devido registro na CVM previsto no art. 19 da Lei nº 6.385/76...,"Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
4,19957-009863-2018-17,Data do julgamento: 11/10/2022,"[M. Invest Planejamento e Administração de Shopping Center S.A., Cyro Santiago Rodrigues, Lorival Rodrigue]","Responsabilidade da M. Invest e de seus administradores pela não elaboração tempestiva e não divulgação de demonstrações financeiras, em infração ...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
5,IA-1997-17,"Rio de Janeiro, 19 de maio de 2005","[Cid Meirelles Ferreira, Luiz Carlos de Lucca, Mérito Auditores Independentes, Milton de Souza Meirelles Filho]","Inobservância do art. 3º, da Instrução CVM nº 10/80. - Aquisição, pela controlada, de ações da controladora em percentual superior ao permitido. I...","Vistos, relatados e discutidos os autos, o Colegiado da CVM, com base na prova dos autos e na legislação aplicável, decidiu, por unanimidade de vo..."
6,IA-2000-06,"Rio de Janeiro, 22 de agosto de 2002","[Sociedade Técnica de Auditoria – SOMATEC S/C, Zilo Matsunaga]",I – Desenvolvimento da atividade de auditoria independente em desacordo com as normas profissionais emanadas do Conselho Federal de Contabilidade....,"Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável dec..."
7,IA-2000-16,NaN,"[IVAN FERREIRA GARCIA, JÚLIO CÉSAR CASSANO, MARCELO PEREIRA QUINTAES, SÉRGIO GUEDES DA COSTA, ALOYSIO LIMA DA SILVEIRA BULCÃO, RONALDO JOSÉ GANEM]",NaN,NaN
8,IA-2003-18,"Rio de Janeiro, 08 de dezembro de 2005","[Allan Humberto de Mello, Alsino de Souza, Apply Auditores Associados S/C, Bruno de Mello Bomeny, Frederico Figueiredo Bomeny, Geraldo Barbosa de ...","Apuração de irregularidades relacionadas à emissão de debêntures destinadas à implementação de empreendimento hoteleiro objeto da companhia, const...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, co..."
9,IA-2004-09,"Rio de Janeiro, 20 de junho de 2006","[Bernardo Clemente da Fonseca Neto, Carlos Ciampolini, Carlos Eduardo Brito de Ávila Camargo, Indusval S.A. CTVM, João Antônio Porchat Forbes, Sér...","

## Teste da veracidade das informações:
Para cada uma das linhas, vamos ver se conseguimos ou não extrair os dados corretamente.

In [26]:
# Linha 1 OK

pd.set_option('display.max_colwidth', None)

df4.iloc[0]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Os dados do df batem com os do pdf.

In [27]:
# Linha 2 OK

df4.iloc[1]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                               19957-005044-2020-15
Data                                                                                                                                                                                                                                                                                                                                                                                                                                                   Data do julgamento: 21/12/2021
Acusados                                                    

In [28]:
# Linha 3 OK

df4.iloc[2]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     19957-006406-2016-09
Data                                        

In [29]:
# Linha 4 OK

df4.iloc[3]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        19957-008445-2016-32
Data                     

In [30]:
# Linha 5 OK

df4.iloc[4]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [31]:
# Linha 6 OK

df4.iloc[5]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [32]:
# Linha 7 OK

df4.iloc[6]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [33]:
# Linha 8 - PROBLEMA NO ARQUIVO

df4.iloc[7]

ID                                                                                                                                                  IA-2000-16
Data                                                                                                                                                       NaN
Acusados    [IVAN FERREIRA GARCIA, JÚLIO CÉSAR CASSANO, MARCELO PEREIRA QUINTAES, SÉRGIO GUEDES DA COSTA, ALOYSIO LIMA DA SILVEIRA BULCÃO, RONALDO JOSÉ GANEM]
Assunto                                                                                                                                                    NaN
Decisão                                                                                                                                                    NaN
Name: 7, dtype: object

In [34]:
# Linha 9 OK

df4.iloc[8]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [35]:
# Linha 10 OK

df4.iloc[9]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       IA-2004-09
Data

In [38]:
# Linha 11 OK

df4.iloc[10]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [39]:
# Linha 12 OK

df4.iloc[11]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [40]:
# Linha 13 - PROBLEMA: NOME DO ÚLTIMO ACUSADO TINHA UMA VÍRGULA NO MEIO E FICOU SEPARADO. COMO RESOLVER?

df4.iloc[12]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [44]:
# Linha 14 OK

df4.iloc[13]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [45]:
# Linha 15 OK

df4.iloc[14]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      RJ-2003-05509
Data              

In [46]:
# Linha 16 OK

df4.iloc[15]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [47]:
# Linha 17 - PROBLEMA ESTÁ NO ARQUIVO (DESFIGURADO)

df4.iloc[16]

ID                   RJ-2008-05752
Data                           NaN
Acusados    [Valayr Hélio Wosiack]
Assunto                        NaN
Decisão                        NaN
Name: 16, dtype: object

In [48]:
# Linha 18 OK

df4.iloc[17]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     RJ-2010-08588
Data                                                                                                                                                                                                                                                                                                                                                                                                                                                          Rio de Janeiro, 14 de dezembro de 2010
Acusados                      

In [49]:
# Linha 19 OK

df4.iloc[18]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      RJ-2016-01742
Data                                                                                                                                                                                                                                                                                                                                                              

In [50]:
# Linha 20 OK

df4.iloc[19]

ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Coisas que vamos ajustar, com base no que vimos:

1. A separação dos acusados por vírgula parece muito rara, mas algumas empresas têm vírgulas no nome. Por isso, não acho que vale a pena usar a vírgula como separador. Vamos alterar isso.

2. A parte de decisão tem alguns problemas: em todos os casos, ela captou perfeitamente o que estava acontecendo e qual foi a decisão tomada de fato. Apesar disso, muitas vezes, é interessante termos os prosentes na sessão, o que viria a um baixo custo. Por isso, acho que vale a pena usar o critério da data como separador principal, e se ela não estiver lá, usamos a expressão "30 dias".

In [70]:
# Função para extrair as informações de cada PDF
def extrair_informacoes_pdf(caminho_pdf):
    nome_arquivo = os.path.basename(caminho_pdf)
    pattern_processo_nome_arquivo = r"(?:\d{5}-\d{6}-\d{4}-\d{2}|IA-\d{4}-\d{2}|RJ-\d{4}-\d{5}|SP-\d{4}-\d{5})"
    match_processo_nome_arquivo = re.search(pattern_processo_nome_arquivo, nome_arquivo)
    numero_processo = match_processo_nome_arquivo.group(0) if match_processo_nome_arquivo else np.nan

    with pdfplumber.open(caminho_pdf) as pdf:
        text = ""
        for i in range(min(2, len(pdf.pages))):
            page = pdf.pages[i]
            text += page.extract_text()

    text = re.sub(r"Extrato de Sessão de Julgamento.*?\n", "", text, flags=re.DOTALL)
    text = re.sub(r"https:\/\/.*\n", "", text)

    pattern_data_julgamento = r"(Data do julgamento:\s*(\d{2}/\d{2}/\d{4})|(?:Rio de Janeiro|São Paulo|Brasília),\s*(\d{2} de (?:janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de 20\d{2}))"
    pattern_acusados = r"(?:Acusados?\s*:|Interessados?\s*:|Indiciados?\s*:|ACUSADOS?\s*:|INTERESSADOS?\s*:|INDICIADOS?\s*:)\s*(.+?)(?=\s*(Assunto|Ementa|Relator|ASSUNTO|EMENTA|RELATOR))"
    pattern_assunto = r"(?:Assunto\s*:|Ementa\s*:|ASSUNTO\s*:|EMENTA\s*:)\s*(.+?)(?=\s*(Decisão|DECISÃO))" 
    pattern_decisao = r"(Vistos, relatados e discutidos os autos\s*(.+?))(?=\s*(?:30 dias|30 (trinta) dias|trinta dias|Participaram desta Sessão|(Rio de Janeiro|São Paulo|Brasília), \d{2} de (janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro) de 20\d{2}))"

    match_data_julgamento = re.search(pattern_data_julgamento, text)
    match_acusados = re.search(pattern_acusados, text, re.DOTALL)
    match_assunto = re.search(pattern_assunto, text, re.DOTALL)
    match_decisao = re.search(pattern_decisao, text, re.DOTALL)

    data_julgamento = match_data_julgamento.group(1).strip() if match_data_julgamento else np.nan
    acusados = match_acusados.group(1).strip() if match_acusados else np.nan
    assunto = " ".join(match_assunto.group(1).strip().splitlines()) if match_assunto else np.nan
    decisao = " ".join(match_decisao.group(1).strip().splitlines()) if match_decisao else np.nan

    return numero_processo, data_julgamento, acusados, assunto, decisao

# Caminho para a pasta com os PDFs
pasta_pdfs = test_folder   # Altere para o caminho da sua pasta

# Criar um dataframe vazio com as colunas apropriadas
df5 = pd.DataFrame(columns=["ID", "Data", "Acusados", "Assunto", "Decisão"])

# Iniciar o timer total
start_time = time.time()

# Loop para processar todos os arquivos PDF na pasta
for arquivo in os.listdir(pasta_pdfs):
    if arquivo.endswith('.pdf'):
        caminho_pdf = os.path.join(pasta_pdfs, arquivo)

        # Timer para cada iteração
        iter_start = time.time()

        # Extrair informações do PDF
        numero_processo, data_julgamento, acusados, assunto, decisao = extrair_informacoes_pdf(caminho_pdf)

        acusados_lista = [acusado.strip() for acusado in acusados.split('\n') if acusado.strip()] if isinstance(acusados, str) else []

        
        nova_linha = pd.DataFrame({
            "ID": [numero_processo],
            "Data": [data_julgamento],
            "Acusados": [acusados_lista],
            "Assunto": [assunto],
            "Decisão": [decisao]
        })

        df5 = pd.concat([df4, nova_linha], ignore_index=True)

        # Timer de cada iteração
        iter_end = time.time()
        iter_time = iter_end - iter_start
        print(f"Tempo para processar {arquivo}: {iter_time:.2f} segundos")

# Calcular o tempo total
end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo total de execução: {execution_time:.2f} segundos")

# Exibir o dataframe com todos os resultados
df5

Tempo para processar 19957-004381-2021-68.pdf: 0.62 segundos
Tempo para processar 19957-005044-2020-15 (RJ-2020-03041).pdf: 0.23 segundos
Tempo para processar 19957-006406-2016-09 (RJ-2016-07486).pdf: 0.41 segundos
Tempo para processar 19957-008445-2016-32 (RJ-2016-08381).pdf: 0.38 segundos
Tempo para processar 19957-009863-2018-17 (RJ-2018-07223).pdf: 0.67 segundos
Tempo para processar IA-1997-17_A.pdf: 0.67 segundos
Tempo para processar IA-2000-06.pdf: 0.55 segundos
Tempo para processar IA-2000-16.pdf: 0.97 segundos
Tempo para processar IA-2003-18.pdf: 0.63 segundos
Tempo para processar IA-2004-09.pdf: 0.93 segundos
Tempo para processar IA-2005-19.pdf: 0.73 segundos
Tempo para processar IA-2006-15.pdf: 0.79 segundos
Tempo para processar RJ-2000-05907.pdf: 0.74 segundos
Tempo para processar RJ-2001-06094.pdf: 0.81 segundos
Tempo para processar RJ-2003-05509.pdf: 0.86 segundos
Tempo para processar RJ-2007-04107.pdf: 0.79 segundos
Tempo para processar RJ-2008-05752.pdf: 0.69 segundos
Te

,ID,Data,Acusados,Assunto,Decisão
0,19957-004381-2021-68,Data do julgamento: 11/04/2023,"[Orla Distribuidora de Títulos e Valores Mobiliários S.A., Paulo Dominguez Landeira, Sagres Investimentos Administração de Recursos Ltda., Expedito Pereira de Araujo Junior, Monetar Distribuidora de Títulos Ltda., Marcelo de Macedo Soares e Silva]","Apurar suposta infração por parte de gestora e de administradoras de FIDC (i) ao art. 15, § 2º, da Instrução CVM n° 356/2001 c/c art. 92, caput, I, da Instrução CVM nº 555/2014; e (ii) ao art. 1º, § 1º, da Instrução CVM nº 444/2006, c/c art. 92, caput, I, da Instrução CVM nº 555/2014. Multas e absolvições.","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, por unanimidade de votos, decidiu: (i) Pela condenação da Orla Distribuidora de Títulos e Valores Mobiliários S.A., à penalidade de multa pecuniária, no valor de R$800.000,00 (oitocentos mil de reais), por infração art. 92, I, da ICVM 555 c/c o art. 1º, §1º, da ICVM 444/2006; (ii) Pela condenação de Paulo Dominguez Landeira, na qualidade de diretor responsável da Orla, à penalidade de multa pecuniária, no valor de R$400.000,00 (quatrocentos mil reais), por infração art. 92, I, da ICVM 555 c/c o art. 1º, §1º, da ICVM 444/2006; (iii) Pela condenação da Sagres Investimentos Administração de Recursos Ltda., à penalidade de multa pecuniária, no valor de R$680.000,00 (seiscentos e oitenta mil reais), por infração art. 92, I, da ICVM 555 c/c o art. 1º, §1º, da ICVM 444/2006; (iv) Pela condenação de Expedito Pereira de Araujo Junior, na qualidade de diretor responsável da Sagres, à penalidade de multa pecuniária, no ICVM 555 c/c o art. 1º, §1º, da ICVM 444/2006; (v) Pela condenação da Monetar Distribuidora de Títulos Ltda., à penalidade de multa pecuniária, no valor de R$680.000,00 (seiscentos e oitenta mil reais), por infração art. 92, I, da ICVM 555 c/c o art. 1º, §1º, da ICVM 444/2006; (vi) Pela condenação de Marcelo de Macedo Soares e Silva, na qualidade de diretor responsável da Monetar, à penalidade de multa pecuniária, no valor de R$340.000,00 (trezentos e quarenta mil reais), por infração art. 92, I, da ICVM 555/2014 c/c o art. 1º, §1º, da ICVM 444/2006; e (vii) Pela absolvição da Orla Distribuidora de Títulos e Valores Mobiliários S.A., da Monetar Distribuidora de Títulos Ltda. e de seus diretores responsáveis, Paulo Dominguez Landeira e Marcelo de Macedo Soares e Silva, da acusação de infração art. 92, I, da ICVM 555/2014 c/c ao art. 15, § 2º, da ICVM 356/2001. Os acusados punidos terão um prazo de"
1,19957-005044-2020-15,Data do julgamento: 21/12/2021,[Maverick Holding S.A.],"Responsabilidade da Maverick Holding S.A., na qualidade de acionista controladora da Mlog S.A., por violação ao art. 106 da Lei nº 6.404/1976. Multa.","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, por unanimidade* de votos, decidiu pela condenação da Maverick, na qualidade de acionista controladora da Mlog, à penalidade de multa pecuniária no valor de R$267.750,00 (duzentos e sessenta e sete mil setecentos e cinquenta reais), por infringir o art. 106 da Lei nº 6.404/1976. O acusado punido terá um prazo de"
2,19957-006406-2016-09,Data do julgamento: 18/06/2019,"[Letícia Ferreira Duarte do Valle, Le Valle Agente Autônomo de Investimentos S/S Ltda.]","Prática de operação fraudulenta no mercado de valores mobiliários. Infração ao item I c/c o item II, letra “c”, da Instrução CVM nº 08/79. Proibição temporária e multa.","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, por unanimidade de votos, decidiu: 1. Com fundamento no art. 11, VII, da Lei nº 6.385/76, aplicar à acusada Leticia Ferreira Duarte do Valle a penalidade de proibição temporária, pelo prazo de cinco anos, de praticar toda e 

In [74]:
pd.set_option('display.max_colwidth', 150)
df5

,ID,Data,Acusados,Assunto,Decisão
0,19957-004381-2021-68,Data do julgamento: 11/04/2023,"[Orla Distribuidora de Títulos e Valores Mobiliários S.A., Paulo Dominguez Landeira, Sagres Investimentos Administração de Recursos Ltda., Expedit...","Apurar suposta infração por parte de gestora e de administradoras de FIDC (i) ao art. 15, § 2º, da Instrução CVM n° 356/2001 c/c art. 92, caput, I...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
1,19957-005044-2020-15,Data do julgamento: 21/12/2021,[Maverick Holding S.A.],"Responsabilidade da Maverick Holding S.A., na qualidade de acionista controladora da Mlog S.A., por violação ao art. 106 da Lei nº 6.404/1976. Multa.","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
2,19957-006406-2016-09,Data do julgamento: 18/06/2019,"[Letícia Ferreira Duarte do Valle, Le Valle Agente Autônomo de Investimentos S/S Ltda.]","Prática de operação fraudulenta no mercado de valores mobiliários. Infração ao item I c/c o item II, letra “c”, da Instrução CVM nº 08/79. Proibiç...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
3,19957-008445-2016-32,Data do julgamento: 18/02/2020,[Pedro Aparecido Ciriello],Realização de oferta pública de contratos de investimento coletivo sem a obtenção do devido registro na CVM previsto no art. 19 da Lei nº 6.385/76...,"Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
4,19957-009863-2018-17,Data do julgamento: 11/10/2022,"[M. Invest Planejamento e Administração de Shopping Center S.A., Cyro Santiago Rodrigues, Lorival Rodrigue]","Responsabilidade da M. Invest e de seus administradores pela não elaboração tempestiva e não divulgação de demonstrações financeiras, em infração ...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, po..."
5,IA-1997-17,"Rio de Janeiro, 19 de maio de 2005","[Cid Meirelles Ferreira, Luiz Carlos de Lucca, Mérito Auditores Independentes, Milton de Souza Meirelles Filho]","Inobservância do art. 3º, da Instrução CVM nº 10/80. - Aquisição, pela controlada, de ações da controladora em percentual superior ao permitido. I...","Vistos, relatados e discutidos os autos, o Colegiado da CVM, com base na prova dos autos e na legislação aplicável, decidiu, por unanimidade de vo..."
6,IA-2000-06,"Rio de Janeiro, 22 de agosto de 2002","[Sociedade Técnica de Auditoria – SOMATEC S/C, Zilo Matsunaga]",I – Desenvolvimento da atividade de auditoria independente em desacordo com as normas profissionais emanadas do Conselho Federal de Contabilidade....,"Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável dec..."
7,IA-2000-16,NaN,"[IVAN FERREIRA GARCIA, JÚLIO CÉSAR CASSANO, MARCELO PEREIRA QUINTAES, SÉRGIO GUEDES DA COSTA, ALOYSIO LIMA DA SILVEIRA BULCÃO, RONALDO JOSÉ GANEM]",NaN,NaN
8,IA-2003-18,"Rio de Janeiro, 08 de dezembro de 2005","[Allan Humberto de Mello, Alsino de Souza, Apply Auditores Associados S/C, Bruno de Mello Bomeny, Frederico Figueiredo Bomeny, Geraldo Barbosa de ...","Apuração de irregularidades relacionadas à emissão de debêntures destinadas à implementação de empreendimento hoteleiro objeto da companhia, const...","Vistos, relatados e discutidos os autos, o Colegiado da Comissão de Valores Mobiliários, com base na prova dos autos e na legislação aplicável, co..."
9,IA-2004-09,"Rio de Janeiro, 20 de junho de 2006","[Bernardo Clemente da Fonseca Neto, Carlos Ciampolini, Carlos Eduardo Brito de Ávila Camargo, Indusval S.A. CTVM, João Antônio Porchat Forbes, Sér...","